## SORT (Simple Online and Real-time Tracking)
https://arxiv.org/abs/1602.00763

- Tracking component를 __Kalman Filter__와 __Hungarian algorithm__과 같은 기술을 이용하여 성능과 속도를 모두 높인 모델


- SORT의 전체적인 구조 
<img src='img/sort.png' width='500'>
<center> https://deep-eye.tistory.com/68 </center>  



In [1]:
from __future__ import print_function

import os
import numpy as np
import matplotlib
matplotlib.use('TkAgg')  # 그래픽 백엔드로 Tk를 사용
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io

import glob
import time
import argparse
from filterpy.kalman import KalmanFilter

In [2]:
np.random.seed(0)

### Hungarian algorithm
- Hungarian algorithm은 작업자를 일에 assign 하는 문제에 대한 optimal한 해를 찾는 알고리즘 (Visual Tracking에서는 이를 이전 프레임의 Object와 현재 프레임의 Object의 Association문제로 볼 수 있음)


#### 예시를 통해 알고리즘 이해하기


__1) 문제의 정의__
- 작업자는 행에 대응하며, 작업은 열에 대응
<img src='img/hungarian.png' width='400'>

__2) 각 행/열에 대하여 최솟값을 모든 행/열에 빼주고, 빼준 값을 오른쪽/아래쪽(marginal 값)에 기입__  

<img src='img/hungarian_2.png' width='800'>
<center> <행 방향 처리>&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<열 방향 처리>  </center>
    
- 1열, 3행의 모든 값이 0임을 알 수 있음
- 0의 값을 가지는 작업자-작업 쌍만 가지고 모든 작업자를 서로 다른 작업에 할당하면 되지만, 위의 행렬에서는 할당할 수 없으므로 아래의 단계를 진행

__3) 행렬에서 가장 작은 값인 1을, 모든 값에서 빼주고(3행 제외), 오른쪽 marginal값에 1 더하기__
<img src='img/hungarian_3.png' width='400'>

__4) 1열에 음수가 생겼으므로, 1열에서 1를 더해주고 아래 marginal값에 -1을 해주기__

<img src='img/hungarian_4.png' width='400'>
- 이 행렬의 모든 원소의 값이 0보다 크거나 값도록 해야 함

__5) 결과__
- 원하는 할당을 구할 수 있음
- 해당 위치에 대한 초기 입력을 사용할 수 있음
<img src='img/hungarian_5.png' width='400'>


__알고리즘 요약__
<img src='img/hungarian_6.png' width='600'>

- 출처 : https://gazelle-and-cs.tistory.com/29

In [3]:
def linear_assigment(cost_matrix):
    '''
    Associate Detections to tracked boxes using Hungarian algorithm
    '''
    try:
        import lap  # Jonker-Volgenant algorithm (faster than hungarian algorithm)
        _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
        return np.array([[y[i], i] for i in x if i >= 0])
    
    except ImportError: # Hungarian algorithm
        from scipy.optimize import linear_sum_assigment
        x, y = linear_sum_assignment(cost_matrix)
        return np.array(list(zip(x, y)))

### Utils

In [4]:
def iou_batch(bb_test, bb_gt):
    '''
    from SORT : Computes IOU between two bboxes in the form [x1, y1, x2, y2]
    '''
    bb_gt = np.expand_dims(bb_gt, 0)     # gt bbox와 test bbox의 개수가 다르기 때문에
    bb_test = np.expand_dims(bb_test, 1) # 서로 다른 축을 expand 해줘서 모든 경우에 대해
                                         # iou 구해주기
    
    '''
    Shape:
        bb_gt - (1, gt_bb_num, 4)
        bb_test - (test_bb_num, 1, 4)
        
        모든 gt bbox와 test bbox 쌍에 대한 iou를 구해주기위해 위와 같이 expand 해줌
    '''
    
    xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
    yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
    xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
    yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
    
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w*h
    o = wh / ((bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])
             + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1]) - wh)
    
    return(o)

In [5]:
def convert_bbox_to_z(bbox):
    '''
    [x1, y1, x2, y2] -> [x, y, s, r] 
    
    x, y : the center of the box
    s : scale(area)
    r : the aspect ratio
    '''
    
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    x = bbox[0] + w/2.
    y = bbox[1] + h/2.
    
    s = w * h    
    r = w / float(h)
    
    return np.array([x, y, s, r]).reshape((4, 1))

In [23]:
def convert_x_to_bbox(x, score=None):
    '''
    [x, y, s, r] -> [x1, y1, x2, y2]
    x1, y1 : the top left
    x2, y2 : the bottom right
    '''
    w = np.sqrt(x[2] * x[3])
    h = x[2] / w
    
    if score == None:
        return np.array([x[0] - w/2., x[1] - h/2., x[0] + w/2., x[1] + h/2.]).reshape((1, 4))
    else:
        return np.array([x[0] - w/2., x[1] - h/2., x[0] + w/2., x[1] + h/2., score]).reshape((1, 5))

### Kalman Filter
: 칼만 필터는 상태 예측(state prediction)과 측정 업데이트(measurement update)를 반복적으로 수행하며, 현재 위치를 계산하는 방법

<img src='img/kalman_filter.png' width='500'>

* 칼만 필터 알고리즘

> 1. 초깃값 설정 : $\hat{x_0}, P_o$  
2. 예측값과 예측 공분산 예측 : $\hat{x_0}^-, {P_k}^-$ (위 첨자 '-'는 예측값을 의미)  
3. 칼만 이득 계산 : $K_k$  
4. 측정값($z_k$)과 예측값의 차이를 보정해서 새로운 추정값 계산 : $\hat{x_k}$  
    ($\hat{x_0}^-$을 prior estimate, $\hat{x_k}$을 posterior estimate라고 하기도 함)
5. 오차 공분산 계산 : $P_k$  

> 
\- $x_t$ : 상태 변수 (The state is stored as a Gaussian (x, P))  
\- $u_t$ : 물체에 가해지는 제어 입력  
\- $z_t$($=Hx_k+v_k$) : 측정값  
\- $v_t$ : 측정 잡음    
\- $w_k$ : 시스템(Process) 잡음 (위의 식에서는 생략되었음. 원래 식에는 $\hat{x_0}^-$를 구하는 식에 +$w_k$항이 있음) 


> \- $R$ : $v_k$(측정 잡음)의 공분산 행렬  
\- $Q$ : $w_k$(시스템 잡음)의 공분산 행렬  
\- $B$ : 사용자 입력에 의한 상태 전이 행렬 (control input matrix)  
\- $H$ : 출력 행렬 (측정값과 상태 변수의 관계를 나타냄)  
\- $A, H, Q, R$은 미리 결정되는 값  

참고자료
- 『칼만 필터는 어렵지 않아』, 김성필 저.
-  https://sharehobby.tistory.com/entry/%EC%B9%BC%EB%A7%8C-%ED%95%84%ED%84%B0Kalman-filter1

In [18]:
class KalmanBoxTracker(object):
    '''
    This class represents the internal state of individual traked objects observed as bbox
    '''
    count = 0
    def __init__(self, bbox):
        '''
        Initialises a tracker using initial bounding box
        
        X(state) : [x, y, s, r, x', y', s'] 
        F : State Transition Matrix
        H : State Mesure Matrix
        R, Q : Noise
        '''
        # constant velocity model (등속모델) 정의
        self.kf = KalmanFilter(dim_x=7, dim_z=4)  # x(state), z(measurement input)
        
        # F : State transition Matrix (A)
        self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
        
        # H : Meature function (측정값과 상태 변수의 관계를 나타냄)
        self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])
        
        # R : mesurement noise의 공분산 행렬
        self.kf.R[2:, 2:] *= 10  
        
        # X의 공분산 행렬
        self.kf.P[4:, 4:] *= 1000 
            # 관측되지 않은 initial velocities(x', y', s')에 높은 uncertainty 부여
        self.kf.P *= 10
        
        # Q : process uncertainty
        self.kf.Q[-1, -1] *= 0.01 
        self.kf.Q[4:, 4:] *= 0.01 # x', y', s'
        
        # Initialize x
        self.kf.x[:4] = convert_bbox_to_z(bbox) 
        
        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1
        
        self.history = []
        self.hits = 0         # 전체 hit
        self.hit_streak = 0   # 연속 hit 기록
        self.age = 0          
        
        
    def update(self, bbox):
        '''
        칼만 필터의 update 단계
        Update the state vector with observed bbox
        '''
        self.time_since_update = 0
        self.history = []
        self.hits += 1
        self.hit_streak += 1  # 연속 hit 기록
        
        # 칼만 필터의 update 단계 
        # compute K
        # update x, P
        self.kf.update(convert_bbox_to_z(bbox))  
        
        
    def predict(self):
        '''
        칼만 필터의 predict 단계 (예측값과 예측 공분산 계산)
        Advance the state vector and returns the predicted bbox estimate
        '''
        
        # 현재 scale(즉, 면적)과 예측 scale의 합이 0보다 작거나 같은 경우 -> 예측 scale을 0으로!
        if ((self.kf.x[6] + self.kf.x[2]) <= 0):  
            self.kf.x[6] *= 0.0
            
        # 칼만 필터의 Predict 단계 (예측값과 예측 공분산 계산)
        # self.kf에 새로운 예측값으로 값이 변경됨
        # x = Fx + Bu   &   P = FPF' + Q
        self.kf.predict()
        
        self.age += 1
        if (self.time_since_update > 0):  # 바로 다음에 update되지 않은 경우, 
            self.hit_streak = 0           # hit_streak(연속 hit 기록)를 0으로 다시 설정해주기
            
        self.time_since_update += 1
        self.history.append(convert_x_to_bbox(self.kf.x))  # 예측값 history에 append
        return self.history[-1]
        
    def get_state(self):
        '''
        현재의 bbox estimate 리턴
        '''
        return convert_to_x_bbos(self.kf.x)
    
    def associate_detections_to_trackers(detection, trackers, iou_threshold=0.3):
        '''
        Assigns detections to tracked object (both represented as bbox)
        Return:
            3 list of matches, unmatched_detections and unmated_trackers
        '''
        
        # 추적하는게 없는 경우 처리
        if(len(tracker) == 0):
            return np.empty((0, 2), dtype=int), np.arange(len(detections)),\
                    np.empty((0, 5), dtype=int)  
            # empty는 이렇게 shape만 정의해줄 경우, 메모리도 초기화되지 않기 때문에
            # 임의의 값이 들어가 있음
            
        
        iou_matrix = iou_batch(detections, trackers)
        
        
        if min(iou_matrix).shape > 0:
            a = (iou_matrix > iou_threshold).astpye(np.int32)
            
            # threshold 보다 큰 것이 하나인 경우
            if a.sum(1).max() == 1 and a.sum(0).max() == 1:
                matched_indices = np.stack(np.where(a), axis=1)
            
            # 1개보다 많을 경우 hungarian algorithm을 이용해 assignment 해주기
            else:
                matched_indices = linear_assigment(-iou_matrix)
        
        else:
            matched_indices = np.empty(shape=(0, 2))
        
        
        unmatched_detections = []
        for d, det in enumerate(detections):
            if d not in matched_indices[:, 0]:
                unmatched_detections.append(d)
        
        unmatched_trackers = []
        for t, trk in enumerate(trackers):
            if t not in matched_indices[:, 1]:
                unmated_trackers.append(t)
                
        # filter out matched with low IOU 
        # 위에서 threshold 넘는게 1개 존재할때만 처리해주었음
        matches = []
        for m in matched_indices:
            if iou_matrix[m[0], m[1]] < iou_threshold:
                unmatched_detections.append(m[0])
                unmatched_trackers.append(m[1])
            else:
                matches.append(m.reshape(1, 2)) 
                
        if len(matches) == 0:
            matches = np.empty((0, 2), dtype=int)
        else:
            matches = np.concatenate(matches, axis=0)  
            # list에 (1, 2)형태 여러개를 0-axis를 기준으로 concatenate
            # ex. [[[1, 2]], [[2, 2]]] -> [[1, 2], [2, 2]]
            
        return matches, np.array(unmatched_detections), np.array(unmatched_trackers)

### Model

In [ ]:
class Sort(object):
    def __init__(self, max_age=1, min_hits=3, iou_threshold=0.3):
        
        
        self.max_age = max_age    # 최대 age (최대 age보다 더 긴 시간동안 update되지 않으면 제거)
        self.min_hits = min_hits  # 판정 기준 - 어느 정도 프레임을 유지해야 포함시킬 수 있음
        self.iou_threshold = iou_threshold 
        self.trackers = []     # KalmanBoxTracker 객체로 이루어진 리스트. 
        self.frame_count = 0   
        
    def update(self, dets=np.empty((0, 5))):
        '''
        Params:
            dets - detections [[x1, y1, x2, y2, score], [x1, y1, x2, y2, score], ..]
        
        Require: this method must be called once for each frame even with empty detections
        Returns a similar array, where the last column is the object ID
        '''
        self.frame_count += 1
        
        # Get predicted locations from existing trackers
        trks = np.zeros((len(self.trackers), 5))  # object 개수
        to_del = []
        ret = []
        
        for t, trk in enumerate(trks):
            pos = self.trackers[t].predict()[0]
            trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]  
                # pos는 예측된 값으로, score는 0으로 초기화
                
            if np.any(np.isna(pos)):  # na값이 하나라도 존재하면 to_del에 append
                to_del.append(t)
                
        trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
            # np.ma.masked_invalid : Na값이나 inf값에 대하여 True, 나머지는 False를 반환
            # compress_rows : 모든 값이 mask=False인 row만!
            # 즉, 모든 값이 Na값이나 inf값이 아닌 row만 trks로 설정
        
        # to_del에 있는 항목들 pop 해주기
        for t in reversed(to_del):
            self.trackes.pop(t)
            
        # dets와 trks associate 해주기
        matched, unmatched_dets, unmatched_trks = \
            associate_detections_to_trackers(dets, trks, self.iou_threshold)
            
        
        # Update matched trackers with assigned detections
        for m in matched:
            self.trackers[m[1]].update(dets[m[0], :])
        
        # Create and initialise new trackers for unmatched detections
        for i in unmatched_dets:
            trk = KalmanBoxTracker(dets[i, :])
            self.trackers.append(trk)
        i = len(self.trackers)
        
        for trk in reversed(self.trackers):
            d = trk.get_state()[0]
            if (trk.time_since_update < 1) and \        # 이전 시점에서도 update 되었고
                (trk.hit_streak >= self.min_hits or \   # 연속 hit가 min_hit 이상
                 self.frame_count <= self.min_hits):    
                ret.append(np.concatenate((d, [trk.id+1])).reshape(1, -1))  
            i -= 1
            
            # remove dead tracklet
            # max_age (최대 나이)보다 time_since_update가 길면 제거
            if (trk.time_since_update > self.max_age):  
                self.trackers.pop(i)                  
                
        if len(ret) > 0:
            return np.concatenate(ret)
        return np.empty((0, 5))
    

### Train

In [1]:
def parse_args():
    '''
    Parse input arguments
    Usage:
        ex) python train.py train --epochs 100 --batch-size 10 
    '''
    parser = argparse.ArgumentParser(description='SORT demo')
    parser.add_argument('--display', dest='display', help='Display online tracker output (slow) [False]',action='store_true')
    parser.add_argument("--seq_path", help="Path to detections.", type=str, default='data')
    parser.add_argument("--phase", help="Subdirectory in seq_path.", type=str, default='train')
    parser.add_argument("--max_age", 
                        help="Maximum number of frames to keep alive a track without associated detections.", 
                        type=int, default=1)
    parser.add_argument("--min_hits", 
                        help="Minimum number of associated detections before track is initialised.", 
                        type=int, default=3)
    parser.add_argument("--iou_threshold", help="Minimum IOU for match.", type=float, default=0.3)
    args = parser.parse_args()
    return args

In [ ]:
if __name__ == '__main__':
    
    # all train
    args = parse_ages()
    display = args.display
    phase = args.phase
    total_time = 0.0
    total_frame = 0
    colours = np.random.rand(32, 3)  # used only for display
    
    if display:
        if not os.path.exists('mot_benchmark'):
            print('\n\tERROR: mot_benchmark link not found!\n\n    Create a symbolic link to the MOT benchmark\n    (https://motchallenge.net/data/2D_MOT_2015/#download). E.g.:\n\n    $ ln -s /path/to/MOT2015_challenge/2DMOT2015 mot_benchmark\n\n')
            exit()
        plt.ion()
        fig = plt.figure()
        ax1 = fig.add_subplot(111, aspect='equal')
        
    if not os.path.exists('output'):
        os.makedir('output')
    
    pattern = os.path.join(args.seq_path, phase, '*', 'det', 'det.txt')
    for seq_dets_fn in glob.glob(pattern):
        # create instance of the Sort tracker
        mot_tracker = Sort(max_age=args.max_age,
                          min_hits=args.min_hits,
                          iou_threshold=args.iou_threshold)  
        seq_dets = np.loadtxt(seq_dets_fn, delimiter=',')
        seq = seq_dets_fn[pattern.find('*'):].split(os.path.seq)[0]
        
        with open(os.path.join('output', '%s.txt'%(seq)), 'w') as out_file:
            print('Processing %s.'%(seq))
            for frame in range(int(seq_dets[:, 0].max())):
                frame += 1  # detection and frame numbers begin at 1
                dets = seq_dets[seq_dets[:, 0] == frame, 2:7]
                dets[:, 2:4] += dets[:, 0:2]  # [x1, y1, w, h] -> [x1, y1, x2, y2]
                total_frames += 1
                
                if display:
                    fn = os.path.join('mot_benchmark', phase, seq, 'img1', '%06d.jpg'%(frame))
                    im = io.imread(fn)
                    ax1.imshow(im)
                    plt.title(seq + ' Tracked Targets')
                
                start_time = time.time()
                trackers = mot_tracker.update(dets)
                cycle_time = time.time() - start_time
                total_time += cycle_time
                
                for d in trackers:
                    print('%d,%d,%.2f,%.2f,%.2f,%.2f,1,-1,-1,-1'%(frame,d[4],d[0],d[1],d[2]-d[0],d[3]-d[1]),file=out_file)
                    if(display):
                        d = d.astype(np.int32)
                        ax1.add_patch(patches.Rectangle((d[0],d[1]),d[2]-d[0],d[3]-d[1],fill=False,lw=3,ec=colours[d[4]%32,:]))
                
                if display:
                    fig.canvas.flush_events()
                    plt.draw()
                    ax1.cla()
        
        print("Total Tracking took: %.3f seconds for %d frames or %.1f FPS" % (total_time, total_frames, total_frames / total_time))

        if(display):
            print("Note: to get real runtime results run without the option: --display")